In [1]:
!pip install Surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357282 sha256=9240156859d06b3b0e89791f8f57073a5944061e3041f7760bf4bdb116e2dcf5
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [20]:
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
import pandas as pd


# Load the dataset
interactions = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Mini project 3 data/interactions_large.csv/interactions_large.csv')
books_metadata = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Mini project 3 data/books_metadata_large.csv/books_metadata_large.csv')

# Define the Reader and load the dataset into Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(interactions[['user_id', 'book_id', 'rating']], reader)

# Use train and test splits from Mini-project 3 (80% train, 20% test)
trainset, testset = train_test_split(data, test_size=0.2)

# Collaborative filtering using SVD (Mini-project 3)
algo_svd = SVD()
algo_svd.fit(trainset)

# SVD Predictions on test set
svd_predictions = algo_svd.test(testset)

# Calculate RMSE for SVD (Mini-project 3)
rmse_svd = accuracy.rmse(svd_predictions)



RMSE: 1.7397


In [22]:
from surprise import Prediction
import pandas as pd



# Step 1: Generate Top-N Popular Books based on ratings count and average rating

top_books = books_metadata[['book_id', 'average_rating', 'ratings_count']]

popular_books = top_books[top_books['ratings_count'] > 100]

popular_books = popular_books.sort_values(by=['average_rating', 'ratings_count'], ascending=False)

N =20

top_n_books = popular_books.head(N)

# Step 2: Hybrid Recommendation Algorithm (SVD + Top-N Popular Books)

# Hybrid predictions: Combine SVD predictions with popular book recommendations
hybrid_predictions = []
for uid, iid, true_r in testset:
    # Get SVD prediction
    svd_pred = algo_svd.predict(uid, iid).est

    # If the book is in the top N popular books, average its rating with the SVD prediction
    if iid in top_n_books['book_id'].values:
        popular_rating = top_n_books[top_n_books['book_id'] == iid]['average_rating'].values[0]
        hybrid_rating = (svd_pred + popular_rating) / 2  # Average of SVD and popular book rating
    else:
        hybrid_rating = svd_pred  # If not in popular books, use only SVD prediction

    hybrid_predictions.append((uid, iid, true_r, hybrid_rating))

# Step 3: Evaluate the Hybrid Model Using RMSE

# Convert hybrid predictions to Surprise's Prediction object format for RMSE evaluation
hybrid_pred_objs = [Prediction(uid, iid, true_r, est, None) for (uid, iid, true_r, est) in hybrid_predictions]

# Calculate RMSE for the hybrid model (Mini-project 4)
rmse_hybrid = accuracy.rmse(hybrid_pred_objs)


# Step 4: Compare RMSE of SVD and Hybrid Models
print(f"SVD RMSE (Mini-project 3): {rmse_svd}")
print(f"Hybrid Model RMSE (Mini-project 4): {rmse_hybrid}")


RMSE: 1.7406
SVD RMSE (Mini-project 3): 1.7397441168331924
Hybrid Model RMSE (Mini-project 4): 1.7405900865650985
